This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

In [2]:
dir_suffix_search = '/home/chansingh/mntv1/suffix_long_rerun'
r_suffix_search = analyze_utils.load_results_and_cache(dir_suffix_search, save_file='r.pkl', only_keep_scalar=False)
r_suffix_search = pd.read_pickle(os.path.join(dir_suffix_search, 'r.pkl'))
r_suffix_search = analyze_utils.postprocess_results(r_suffix_search,   )
r = r_suffix_search
r.to_pickle('../results/prompt_gen/results_suffix_all.pkl')
r = pd.read_pickle('../results/prompt_gen/results_suffix_all.pkl')

 72%|███████▎  | 29/40 [00:39<00:09,  1.15it/s]

skipping 885d0a8f6f08d429faf8334b23a057df4b922b421e03740d20e34464f95bbe5csuvv


100%|██████████| 40/40 [00:42<00:00,  1.07s/it]


# Look at top suffix candidates

In [4]:
r0 = r[(r.max_num_tokens == 6) * (r.n_shots == 1) * (r.checkpoint == 'EleutherAI/gpt-j-6B')]
print(r0.shape)
print(list(r0.keys()))

(19, 56)
['task_name', 'n_shots', 'max_dset_size', 'max_digit', 'template_num_init_string', 'template_num_task_phrasing', 'train_split_frac', 'checkpoint', 'max_num_tokens', 'beam_size', 'beam_size_extra', 'use_single_query', 'use_stopwords', 'use_early_stopping', 'use_generic_query', 'float16', 'seed', 'save_dir', 'use_cpu_only', 'use_parallelformers', 'use_cache', 'use_verbose_saving', 'epoch_save_interval', 'batch_size', 'task_name_list', 'suffix_str_init', 'len_suffix_str_init', 'suffix_str_added', 'num_tokens_added', 'num_model_queries', 'running_prob', 'final_answer_pos_initial_token', 'final_answer_full', 'final_answer_added', 'final_model_queries', 'final_num_suffixes_checked', 'final_answer_depth', 'top_prompt', 'top_candidates', 'top_probs', 'final_answer_found', 'Recall @ 3 suffixes', 'Recall @ 5 suffixes', 'Recall @ 10 suffixes', 'Recall @ 15 suffixes', 'Recall @ 20 suffixes', 'Recall @ 25 suffixes', 'Recall @ 30 suffixes', 'Recall @ 40 suffixes', 'Recall @ 50 suffixes', 'R

In [8]:
r0[['task_name', 'top_prompt']]

,task_name,top_prompt
1,double_one,f(x).\n\n
2,max_two,the sum of the input numbers
3,task1147_country_currency,the string representation of the input
5,task1191_food_veg_nonveg,the input.\n\n<|endoftext|>
6,add_two,the sum of the input numbers
7,multiply_two,the sum of the input numbers
10,task1146_country_capital,the name of the capital of
11,subtract_two,-1.\n\n<|endoftext|>
15,square_one,f(x).\n\n
16,divide_two,the result of the division.


The most relevant keys are top_prompt, top_candidates, top_probs.

The relevant keys for metrics are reciprocal_rank, reciprocal_rank_multi.


**Save out top prompts**

In [9]:
results_acc_dir = oj('/home/chansingh/interpretable-autoprompting', 'results', 'generalization_acc')
prompts = pkl.load(open(oj(results_acc_dir, 'prompts_all.pkl'), 'rb'))

In [10]:
idxs = prompts.index
def get_top_prompt(r0, task_name):
    top_prompt = r0[r0['task_name'] == task_name]['top_prompt']
    if len(top_prompt) == 0:
        return np.nan
    else:
        return top_prompt
prompts_suff = [get_top_prompt(r0, task_name) for task_name in idxs]
# prompts['suffix'] = 

In [12]:
for i in enumerate(idxs):
    print(i, idxs[i], prompts_suff[i])

TypeError: 'Index' object cannot be interpreted as an integer